In [1]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import numpy as np
import ampligraph

## Train and predict scores

In [3]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# create the model with transe scoring function
partitioned_model = ScoringBasedEmbeddingModel(eta=2, 
                                               k=50, 
                                               scoring_type='TransE')
partitioned_model.compile(optimizer='adam', loss='multiclass_nll')

# Here we have specified the path of the input file
# you can also load using default dataloaders load_fb15k_237() and pass numpy array inputs
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                      batch_size=10000, 
                      partitioning_k=3, # set flag to partition the inputs
                      epochs=10)


_split: memory before: 864.0Bytes, after: 13.002MB, consumed: 13.001MB; exec time: 138.4s
Epoch 1/10
32/32 [==============================] - 2s 60ms/step - loss: 9643.8486
Epoch 2/10
32/32 [==============================] - 1s 38ms/step - loss: 9608.9883
Epoch 3/10
32/32 [==============================] - 1s 39ms/step - loss: 9564.9219
Epoch 4/10
32/32 [==============================] - 1s 39ms/step - loss: 9509.7002
Epoch 5/10
32/32 [==============================] - 1s 40ms/step - loss: 9443.8955
Epoch 6/10
32/32 [==============================] - 1s 40ms/step - loss: 9366.0791
Epoch 7/10
32/32 [==============================] - 1s 40ms/step - loss: 9276.2832
Epoch 8/10
32/32 [==============================] - 1s 40ms/step - loss: 9176.0684
Epoch 9/10
32/32 [==============================] - 1s 39ms/step - loss: 9067.4639
Epoch 10/10
32/32 [==============================] - 1s 39ms/step - loss: 8952.7305


In [4]:
# Unfiltered evaluation
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



28 triples containing invalid keys skipped!
53/53 [==============================] - 221s 4s/step


(964.4983853606028, 0.08503312432917728, 0.0, 0.22903415206967415, 20438)

In [5]:
# Filtered evaluation
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                        batch_size=400,
                        corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
53/53 [==============================] - 413s 8s/step


(835.887488991095,
 0.18460814776765044,
 0.1265779430472649,
 0.30357667090713375,
 20438)

In [6]:
from ampligraph.utils import save_model
save_model(model=partitioned_model, model_name_path='./partitioned_model')

WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


In [7]:
from ampligraph.utils import restore_model
model = restore_model('./partitioned_model')

In [8]:
# Unfiltered evaluation
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!
53/53 [==============================] - 230s 4s/step


(964.4983853606028, 0.08503312432917728, 0.0, 0.22903415206967415, 20438)

In [9]:
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                        batch_size=400,
                        corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
53/53 [==============================] - 406s 8s/step


(835.887488991095,
 0.18460814776765044,
 0.1265779430472649,
 0.30357667090713375,
 20438)